In [12]:
import tensorflow as tf
import pickle
import numpy as np

In [13]:
pickle_file = '../input_data/notMnist/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (2000, 28, 28) (2000,)
Validation set (100, 28, 28) (100,)
Test set (100, 28, 28) (100,)


In [14]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (2000, 28, 28, 1) (2000, 10)
Validation set (100, 28, 28, 1) (100, 10)
Test set (100, 28, 28, 1) (100, 10)


In [15]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

In [16]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
    # Input data.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(
        tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        pool_1 = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        conv = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
        hidden_1 = tf.nn.relu(conv + layer2_biases)
        pool_2 = tf.nn.max_pool(hidden_1, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        
        shape = pool_2.get_shape().as_list()
        
        reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
        
        return tf.matmul(hidden, layer4_weights) + layer4_biases

    # Training computation.
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [18]:
num_steps = 30001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' %
                  accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.670720
Minibatch accuracy: 0.0%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.286016
Minibatch accuracy: 12.5%
Validation accuracy: 25.0%
Minibatch loss at step 100: 1.400930
Minibatch accuracy: 31.2%
Validation accuracy: 59.0%
Minibatch loss at step 150: 1.911180
Minibatch accuracy: 43.8%
Validation accuracy: 64.0%
Minibatch loss at step 200: 0.903042
Minibatch accuracy: 75.0%
Validation accuracy: 70.0%
Minibatch loss at step 250: 1.284542
Minibatch accuracy: 50.0%
Validation accuracy: 74.0%
Minibatch loss at step 300: 0.571855
Minibatch accuracy: 87.5%
Validation accuracy: 77.0%
Minibatch loss at step 350: 0.134088
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Minibatch loss at step 400: 0.626291
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 450: 0.653610
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 500: 0.535406
Minibatch accuracy: 81.2%
Validation accuracy: 76.0%
M

Minibatch loss at step 4500: 0.000255
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4550: 0.000082
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4600: 0.000182
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 4650: 0.001442
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4700: 0.002754
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4750: 0.000929
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4800: 0.001060
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4850: 0.000204
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 4900: 0.000079
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 4950: 0.000008
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 5000: 0.001560
Minibatch accuracy: 100.0%
Validation accu

Minibatch loss at step 9000: 0.000384
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 9050: 0.000253
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 9100: 0.000285
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 9150: 0.000321
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 9200: 0.000508
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 9250: 0.000053
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 9300: 0.000332
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 9350: 0.000036
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 9400: 0.000030
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 9450: 0.000517
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 9500: 0.000346
Minibatch accuracy: 100.0%
Validation accu

Minibatch loss at step 13450: 0.000220
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 13500: 0.000093
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 13550: 0.000177
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 13600: 0.000090
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 13650: 0.000060
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 13700: 0.000101
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 13750: 0.000162
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 13800: 0.000043
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 13850: 0.000012
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 13900: 0.000034
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 13950: 0.000337
Minibatch accuracy: 100.0%
Vali

Minibatch loss at step 17900: 0.000074
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 17950: 0.000070
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18000: 0.000061
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18050: 0.000079
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18100: 0.000139
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18150: 0.000046
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18200: 0.000054
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18250: 0.000177
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18300: 0.000099
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18350: 0.000079
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 18400: 0.000078
Minibatch accuracy: 100.0%
Vali

Minibatch loss at step 22350: 0.000021
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 22400: 0.000168
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 22450: 0.000099
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 22500: 0.000167
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 22550: 0.000020
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 22600: 0.000020
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 22650: 0.000200
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 22700: 0.000075
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 22750: 0.000061
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 22800: 0.000054
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 22850: 0.000075
Minibatch accuracy: 100.0%
Vali

Minibatch loss at step 26800: 0.000034
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 26850: 0.000102
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 26900: 0.000011
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 26950: 0.000038
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 27000: 0.000104
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 27050: 0.000004
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 27100: 0.000050
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 27150: 0.000031
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 27200: 0.000040
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 27250: 0.000042
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 27300: 0.000037
Minibatch accuracy: 100.0%
Vali